## 앙상블 학습

In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

import time
import warnings
warnings.filterwarnings('ignore')

# 데이터 불러오기
cancer = load_breast_cancer()
df = pd.read_csv('./diabetes.csv')

X = df.drop(['Outcome'], axis=1)
y = df['Outcome']

다음 실습을 통해 하드 보팅과 소프트 보팅의 정확도를 비교해보고, 앙상블 방식이 아닌 개별 분류기의 예측 성능 또한 출력해서 비교

사용할 개별 분류기:
1. 로지스틱 회귀
2. K 최근접 이웃
3. 랜덤 포레스트

In [4]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

# 성능 평가 함수
def get_clf_eval(y_test, pred=None):
  confusion = confusion_matrix(y_test, pred)
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  print('오차 행렬')
  print(confusion)
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy, precision, recall))

In [5]:
# 개별 모델 생성
# 로지스틱 회귀의 solver='liblinear' 설정
# K 최근접 이웃의 n_neighbors=8 설정
# 랜덤포레스트의 random_state=11 고정

lr = LogisticRegression(solver='liblinear')
knn = KNeighborsClassifier(n_neighbors=8)
rf = RandomForestClassifier(random_state=11)

# 데이터 세트 분리
# test_size=0.2, random_state=11로 설정해줍니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

# 하드 보팅 기반의 앙상블 모델로 구현한 분류기 생성
vo_hard = VotingClassifier(estimators=[('LR', lr), ('KNN', knn), ('RF', rf)], voting='hard')

# Hard Voting Classifier 학습/예측/평가
# 평가는 get_clf_eval로 출력해주세요
vo_hard.fit(X_train, y_train)
pred_h = vo_hard.predict(X_test)
print('하드 보팅')
get_clf_eval(y_test, pred_h)

# 소프트 보팅 기반의 앙상블 모델로 구현한 분류기 생성
vo_soft = VotingClassifier(estimators=[('LR', lr), ('KNN', knn), ('RF', rf)], voting='soft')

# Soft Voting Classifier 학습/예측/평가
# 평가는 get_clf_eval로 출력해주세요
vo_soft.fit(X_train, y_train)
pred_s = vo_soft.predict(X_test)
print('소프트 보팅')
get_clf_eval(y_test, pred_s)

하드 보팅
오차 행렬
[[90 10]
 [29 25]]
정확도: 0.7468, 정밀도: 0.7143, 재현율: 0.4630
소프트 보팅
오차 행렬
[[86 14]
 [26 28]]
정확도: 0.7403, 정밀도: 0.6667, 재현율: 0.5185


In [6]:
# 개별 모델의 학습/예측/평가 (for문 사용)
# 세 가지 분류기를 리스트 형태로 저장
classifiers = [lr, knn, rf]

# for문을 사용하여 분류기 이름과 해당 분류기를 사용했을 때의 get_clf_eval 출력
for classifier in classifiers:
  classifier.fit(X_train, y_train)
  pred = classifier.predict(X_test)
  class_name = classifier.__class__.__name__
  print(class_name)
  get_clf_eval(y_test, pred)

LogisticRegression
오차 행렬
[[90 10]
 [31 23]]
정확도: 0.7338, 정밀도: 0.6970, 재현율: 0.4259
KNeighborsClassifier
오차 행렬
[[88 12]
 [31 23]]
정확도: 0.7208, 정밀도: 0.6571, 재현율: 0.4259
RandomForestClassifier
오차 행렬
[[85 15]
 [28 26]]
정확도: 0.7208, 정밀도: 0.6341, 재현율: 0.4815


### 랜덤 포레스트

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=0)

아래의 코드를 실행시켜 주세요.

In [8]:
rf_clf = RandomForestClassifier(random_state=0)

rf_clf.fit(X_train,y_train)
pred = rf_clf.predict(X_test)

accuracy = accuracy_score(y_test,pred)

print('랜덤 포레스트 정확도 : {0:.4f}'.format(accuracy))

랜덤 포레스트 정확도 : 0.7857


#### 정확도 개선

In [9]:
# RandomForestClassifier 객체를 생성
rf_clf = RandomForestClassifier(random_state=0,max_depth=8,n_estimators=100,min_samples_leaf=10)

# rf_clf를 이용하여 랜덤 포레스트 학습 및 예측 성능을 평가
rf_clf.fit(X_train,y_train)
pred = rf_clf.predict(X_test)

# 정확도를 accuracy변수에 저장
accuracy = accuracy_score(y_test,pred)

print('랜덤 포레스트 정확도 : {0:.4f}'.format(accuracy))

랜덤 포레스트 정확도 : 0.8312


X,y를 train_test_split() 함수로 학습 데이터와 테스트 데이터로 분류

In [10]:
# X_features와 y_label을 test_size = 0.2로 분리하고 random_state는 156으로 지정
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 156)

### Adaboost으로 학습


- n_estimators=50, learning_rate=1로 지정합니다.

In [12]:
# 시작 시간 설정
start_time = time.time()

# Adaboost 학습 및 예측
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1)
abc.fit(X_train, y_train)
pred = abc.predict(X_test)

# Adaboost 정확도와 수행시간 출력
print('Adaboost 정확도:', np.round(accuracy_score(y_test, pred),4))
print('Adaboost 수행 시간: {0:.3f}초'.format(time.time() - start_time))


Adaboost 정확도: 0.7403
Adaboost 수행 시간: 0.216초


#### GBM으로 학습하여 정확도와 GBM 수행 시간을 측정


In [13]:
from sklearn.model_selection import GridSearchCV

# 시작 시간 설정
start_time = time.time()

# GBM 학습 및 예측
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)

# n_estimators 와 learning_rate는 각각 자유롭게 2개씩 설정하여 params 변수에 딕셔너리 형태로 지정하세요.
params = {
    'n_estimators':[100,500],
    'learning_rate':[0.05, 0.1]
}

# GridSearchCV를 적용하여 하이퍼 파라미터를 최적화해봅니다.
grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)

# GMB 최적 하이퍼 파라미터, 최고 예측 정확도와 수행시간을 순서대로 출력하세요.
print('GBM 최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('GBM 최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))
print('GBM 수행 시간: {0:.3f}초'.format(time.time() - start_time))

# 최적으로 학습된 estimator로 예측 수행하고 정확도를 출력하세요.
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도: {0:.4f}'.format(gb_accuracy))


Fitting 2 folds for each of 4 candidates, totalling 8 fits
GBM 최적 하이퍼 파라미터:
 {'learning_rate': 0.1, 'n_estimators': 100}
GBM 최고 예측 정확도: 0.7573
GBM 수행 시간: 3.073초
GBM 정확도: 0.7597


### XGBoost

아래 데이터를 로드

In [14]:
import pandas as pd
import xgboost as xgb
from xgboost import plot_importance
import numpy as np
from sklearn. model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

diabetes = pd.read_csv('diabetes.csv')

features = diabetes.iloc[:,:-1]
label = diabetes.iloc[:,-1]

파이썬 래퍼 XGBoost로 학습하여 나온 예측 확률 값 출력(10개씩)

In [15]:
# X_features와 y_label을 test_size = 0.2로 분리하고, tarin 데이터에서 10%는 검증용으로 분리하기
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size = 0.2, random_state = 156)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 156)

# DMatrix 생성
dtr = xgb.DMatrix(data = X_tr, label = y_tr)
dval = xgb.DMatrix(data = X_val, label = y_val)
dtest = xgb.DMatrix(data = X_test, label = y_test)

# 파라미터 지정
params = {'max_depth':3, 'eta':0.05, 'objective':'binary:logistic', 'eval_metric':'logloss'}

# XGBoost를 이용하여 학습(부스팅 반복 횟수는 300회, 조기 중단 최소 반복 횟수는 50회)
xgb_model = xgb.train(params = params, dtrain = dtr, num_boost_round = 300, early_stopping_rounds = 50, evals = [(dtr,'train'),(dval,'eval')])

# XGBoost로 학습하여 나온 예측 확률 값과 예측값을 출력
pred_probs = xgb_model.predict(dtest)
print("예측 확률 값 :", np.round(pred_probs[:10],4))
preds = [1 if x >= 0.5 else 0 for x in pred_probs]
print("예측 값 :", preds[:10])


[0]	train-logloss:0.61462	eval-logloss:0.76101
[1]	train-logloss:0.60162	eval-logloss:0.74479
[2]	train-logloss:0.58980	eval-logloss:0.73108
[3]	train-logloss:0.57752	eval-logloss:0.71674
[4]	train-logloss:0.56627	eval-logloss:0.70301
[5]	train-logloss:0.55592	eval-logloss:0.69129
[6]	train-logloss:0.54584	eval-logloss:0.68073
[7]	train-logloss:0.53703	eval-logloss:0.67041
[8]	train-logloss:0.52829	eval-logloss:0.66225
[9]	train-logloss:0.52101	eval-logloss:0.65626
[10]	train-logloss:0.51344	eval-logloss:0.64967
[11]	train-logloss:0.50706	eval-logloss:0.64293
[12]	train-logloss:0.50001	eval-logloss:0.63546
[13]	train-logloss:0.49349	eval-logloss:0.62863
[14]	train-logloss:0.48746	eval-logloss:0.62240
[15]	train-logloss:0.48163	eval-logloss:0.61695
[16]	train-logloss:0.47580	eval-logloss:0.61113
[17]	train-logloss:0.47109	eval-logloss:0.60880
[18]	train-logloss:0.46594	eval-logloss:0.60301
[19]	train-logloss:0.46116	eval-logloss:0.59857
[20]	train-logloss:0.45693	eval-logloss:0.59576
[2

사이킷런 래퍼 XGBoost로 학습하여 나온 예측값을 10개 출력

In [16]:
from xgboost import XGBClassifier

xgb_wrapper = XGBClassifier(n_estimators = 300, learning_rate = 0.05, max_depth = 3, eval_metric = 'logloss')
xgb_wrapper.fit(X_tr, y_tr, early_stopping_rounds = 50, eval_set = [(X_tr, y_tr),(X_val, y_val)], verbose = True)

xgb_preds = xgb_wrapper.predict(X_test)
print("예측 확률 값 :", np.round(xgb_preds[:10],4))


[0]	validation_0-logloss:0.61462	validation_1-logloss:0.76101
[1]	validation_0-logloss:0.60162	validation_1-logloss:0.74479
[2]	validation_0-logloss:0.58980	validation_1-logloss:0.73108
[3]	validation_0-logloss:0.57752	validation_1-logloss:0.71674
[4]	validation_0-logloss:0.56627	validation_1-logloss:0.70301
[5]	validation_0-logloss:0.55592	validation_1-logloss:0.69129
[6]	validation_0-logloss:0.54584	validation_1-logloss:0.68073
[7]	validation_0-logloss:0.53703	validation_1-logloss:0.67041
[8]	validation_0-logloss:0.52829	validation_1-logloss:0.66225
[9]	validation_0-logloss:0.52101	validation_1-logloss:0.65626
[10]	validation_0-logloss:0.51344	validation_1-logloss:0.64967
[11]	validation_0-logloss:0.50706	validation_1-logloss:0.64293
[12]	validation_0-logloss:0.50001	validation_1-logloss:0.63546
[13]	validation_0-logloss:0.49349	validation_1-logloss:0.62863
[14]	validation_0-logloss:0.48746	validation_1-logloss:0.62240
[15]	validation_0-logloss:0.48163	validation_1-logloss:0.61695
[1